# Candlestick Chart Visualizer with Plotly

This notebook demonstrates how to visualize candlestick charts and add interactive lines and dots using Plotly. You can load OHLCV data, plot candlesticks, and annotate the chart with lines and markers.

In [1]:
# Multi-File Chart Visualizer Function
import pandas as pd
import json
import plotly.graph_objects as go
from src.ta.technical_analysis import TechnicalAnalysisProcessor
from src.ta.middlewares.zigzag import zigzag_middleware
from src.ta.middlewares.volume_profile import volume_profile_middleware
from src.ta.middlewares.channels import channels_middleware
from src.ta.middlewares.levels import levels_middleware

def load_chart_data(file_path):
    """
    Load chart data from a JSON file and return processed DataFrame with metadata
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    symbol = data['symbolName']
    time_frame = data['interval']
    df = pd.DataFrame(data['candles'])
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    
    return {
        'symbol': symbol,
        'time_frame': time_frame,
        'df': df,
        'file_path': file_path
    }

def visualize_chart(chart_data, middlewares=[], useLogScale=True):
    """
    Visualize a chart from loaded data
    
    Args:
        chart_data: Dictionary with keys: 'symbol', 'time_frame', 'df', 'file_path'
        middlewares: List of middleware functions to register (default: all middlewares)
        useLogScale: Whether to use logarithmic scale
    """
    symbol = chart_data['symbol']
    time_frame = chart_data['time_frame']
    df = chart_data['df'].copy()
    
    # Default middlewares if none provided
    
    print(f"Processing {len(df)} candles for {symbol} at {time_frame} interval")
    print(f"   Middlewares: {[m.__name__ for m in middlewares]}")
    
    # Run TA processor
    processor = TechnicalAnalysisProcessor(df, time_frame, useLogScale)
    
    # Register middlewares
    for middleware in middlewares:
        processor.register_middleware(middleware)
    
    analysis = processor.run()
    df = processor.df
    
    # Create figure
    fig = go.Figure(data=[
        go.Candlestick(
            x=df.index,
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close'],
            name='Candlestick',
            showlegend=False
        )
    ])
    
    # Draw zigzag lines if present
    if 'zigzag' in analysis:
        for line in analysis['zigzag'].get('lines', []):
            if isinstance(line, tuple) and len(line) == 3:
                (t1, p1, _), (t2, p2, _), line_type = line
                fig.add_trace(go.Scatter(x=[t1, t2], y=[p1, p2], mode='lines', line=dict(color='blue', width=2), name='ZigZag', showlegend=False))
        for pivot in analysis['zigzag'].get('pivots', []):
            if len(pivot) == 3:
                t, p, typ = pivot
                fig.add_trace(go.Scatter(x=[t], y=[p], mode='markers', marker=dict(color='red', size=10), name=f'Pivot {typ}', showlegend=False))
    
    # Draw channel lines if present
    if 'channels' in analysis:
        for line in analysis['channels'].get('lines', []):
            if isinstance(line, tuple) and len(line) == 3:
                (t1, p1, _), (t2, p2, _), line_type = line
                fig.add_trace(go.Scatter(x=[t1, t2], y=[p1, p2], mode='lines', line=dict(color='orange', width=2, dash='dash'), name=line_type, showlegend=True))
    
    # Draw channel lines if present
    if 'levels' in analysis:
        for line in analysis['levels'].get('lines', []):
            if isinstance(line, tuple) and len(line) == 3:
                (t1, p1, _), (t2, p2, _), line_type = line
                fig.add_trace(go.Scatter(x=[t1, t2], y=[p1, p2], mode='lines', line=dict(color='orange', width=2, dash='dash'), name=line_type, showlegend=True))

    # Draw volume profile as boxes if present
    if 'volume_profile' in analysis and 'vp' in analysis['volume_profile']:
        for vp_item in analysis['volume_profile']['vp']:
            if isinstance(vp_item, tuple) and len(vp_item) == 3:
                bin_tuple, bin_vol, norm_vol = vp_item
                if isinstance(bin_tuple, tuple) and len(bin_tuple) == 2:
                    bin_start, bin_end = bin_tuple
                    x0 = df.index[0]
                    bar_length = int(norm_vol * len(df.index))
                    x1 = df.index[min(bar_length, len(df.index)-1)]
                    fig.add_shape(
                        type='rect',
                        x0=x0,
                        x1=x1,
                        y0=bin_start,
                        y1=bin_end,
                        line=dict(color='gold', width=1, dash='dot'),
                        fillcolor='rgba(255,215,0,0.2)'
                    )
    
    # Draw volume profile POC, VAH, VAL levels if present
    if 'volume_profile_periods' in analysis and 'lines' in analysis['volume_profile_periods']:
        for line in analysis['volume_profile_periods']['lines']:
            if isinstance(line, tuple) and len(line) == 3:
                (t1, p1, _), (t2, p2, _), line_type = line
        
                if 'naked_poc_line' in line_type:
                    color = 'white'
                    name = 'nPOC'
                elif 'poc_line' in line_type:
                    color = 'white'
                    name = 'POC'
                elif 'VAH' in line_type:
                    color = 'green'
                    name = 'VAH'
                elif 'VAL' in line_type:
                    color = 'red' 
                    name = 'VAL'
                else:
                    color = 'deepskyblue'
                    name = 'Volume Level'
                
                fig.add_trace(go.Scatter(x=[t1, t2], y=[p1, p2], mode='lines', line=dict(color=color, width=1, dash='dash'), name=name, showlegend=True))

    
    # Update layout
    fig.update_layout(
        title=f'{symbol} - {time_frame} - TA Chart Visualizer', 
        xaxis_title='Time', 
        yaxis_title='Price', 
        template='plotly_dark', 
        yaxis_type='log' if useLogScale else 'linear',
        height=800,
        xaxis=dict(fixedrange=False),
        yaxis=dict(fixedrange=False),
        showlegend=False,  # Remove legend
        xaxis_rangeslider_visible=False  # Remove range slider at bottom
    )
    
    fig.show()
    return fig

print("✅ Chart data loader and visualizer functions created!")


✅ Chart data loader and visualizer functions created!


In [2]:
# Function to split chart data into segments
def split_chart_data(chart_data, num_segments=10):
    """
    Split chart data into multiple segments
    
    Args:
        chart_data: Dictionary with keys: 'symbol', 'time_frame', 'df', 'file_path'
        num_segments: Number of segments to split into (default: 10)
    
    Returns:
        List of chart_data dictionaries, each with a portion of the candles
    """
    df = chart_data['df']
    total_candles = len(df)
    segment_size = total_candles // num_segments
    
    segments = []
    for i in range(num_segments):
        start_idx = i * segment_size
        
        # Last segment gets any remaining candles
        if i == num_segments - 1:
            end_idx = total_candles
        else:
            end_idx = (i + 1) * segment_size
        
        segment_df = df.iloc[start_idx:end_idx].copy()
        
        segment_data = {
            'symbol': chart_data['symbol'],
            'time_frame': chart_data['time_frame'],
            'df': segment_df,
            'file_path': chart_data['file_path'],
            'segment': i + 1,
            'total_segments': num_segments,
            'start_date': segment_df.index[0],
            'end_date': segment_df.index[-1]
        }
        
        segments.append(segment_data)
    
    return segments

print("✅ Chart data splitter function created!")


✅ Chart data splitter function created!


In [3]:
# Load data from files first
file_paths = [
    #'./data/BTCUSDT-1h.json',
    './data/BTCUSDT-M.json', 
    './data/BTCUSDT-W.json',
    './data/BTCUSDT-D.json'
]

# Split charts into 10 segments and visualize each
num_segments = 10

# Load all chart data
chart_data_list = []
for file_path in file_paths:
    try:
        print(f"📂 Loading: {file_path}")
        chart_data = load_chart_data(file_path)
        chart_data_list.append(chart_data)
        print(f"   ✅ Loaded {chart_data['symbol']} - {chart_data['time_frame']} ({len(chart_data['df'])} candles)")
    except Exception as e:
        print(f"   ❌ Error loading {file_path}: {e}")

print(f"\n✅ Total charts loaded: {len(chart_data_list)}")

# Define which middlewares to use (you can customize this list)
my_middlewares = [
    #zigzag_middleware,
    #volume_profile_middleware,
    #channels_middleware,
    levels_middleware
]


for chart_data in chart_data_list:
    print(f"\n{'='*70}")
    print(f"🔪 Splitting {chart_data['symbol']} - {chart_data['time_frame']} into {num_segments} segments")
    print('='*70)
    
    # Split the chart data into segments
    segments = split_chart_data(chart_data, num_segments=num_segments)
    
    print(f"✅ Created {len(segments)} segments:")
    for seg in segments:
        print(f"   Segment {seg['segment']}/{seg['total_segments']}: "
              f"{seg['start_date'].strftime('%Y-%m-%d')} to {seg['end_date'].strftime('%Y-%m-%d')} "
              f"({len(seg['df'])} candles)")
    
    # Visualize each segment
    for segment_data in segments:
        try:
            print(f"\n{'='*70}")
            print(f"📊 Visualizing Segment {segment_data['segment']}/{segment_data['total_segments']}: "
                  f"{segment_data['symbol']} - {segment_data['time_frame']}")
            print(f"   Period: {segment_data['start_date'].strftime('%Y-%m-%d %H:%M')} to "
                  f"{segment_data['end_date'].strftime('%Y-%m-%d %H:%M')}")
            print('='*70)
            
            visualize_chart(segment_data, middlewares=my_middlewares, useLogScale=True)
            
        except Exception as e:
            print(f"❌ Error visualizing segment {segment_data['segment']}: {e}")
            import traceback
            traceback.print_exc()


📂 Loading: ./data/BTCUSDT-M.json
   ✅ Loaded BTCUSDT - M (68 candles)
📂 Loading: ./data/BTCUSDT-W.json
   ✅ Loaded BTCUSDT - W (288 candles)
📂 Loading: ./data/BTCUSDT-D.json
   ✅ Loaded BTCUSDT - D (2021 candles)

✅ Total charts loaded: 3

🔪 Splitting BTCUSDT - M into 10 segments
✅ Created 10 segments:
   Segment 1/10: 2020-03-01 to 2020-08-01 (6 candles)
   Segment 2/10: 2020-09-01 to 2021-02-01 (6 candles)
   Segment 3/10: 2021-03-01 to 2021-08-01 (6 candles)
   Segment 4/10: 2021-09-01 to 2022-02-01 (6 candles)
   Segment 5/10: 2022-03-01 to 2022-08-01 (6 candles)
   Segment 6/10: 2022-09-01 to 2023-02-01 (6 candles)
   Segment 7/10: 2023-03-01 to 2023-08-01 (6 candles)
   Segment 8/10: 2023-09-01 to 2024-02-01 (6 candles)
   Segment 9/10: 2024-03-01 to 2024-08-01 (6 candles)
   Segment 10/10: 2024-09-01 to 2025-10-01 (14 candles)

📊 Visualizing Segment 1/10: BTCUSDT - M
   Period: 2020-03-01 00:00 to 2020-08-01 00:00
Processing 6 candles for BTCUSDT at M interval
   Middlewares: ['


📊 Visualizing Segment 2/10: BTCUSDT - M
   Period: 2020-09-01 00:00 to 2021-02-01 00:00
Processing 6 candles for BTCUSDT at M interval
   Middlewares: ['levels_middleware']



📊 Visualizing Segment 3/10: BTCUSDT - M
   Period: 2021-03-01 00:00 to 2021-08-01 00:00
Processing 6 candles for BTCUSDT at M interval
   Middlewares: ['levels_middleware']



📊 Visualizing Segment 4/10: BTCUSDT - M
   Period: 2021-09-01 00:00 to 2022-02-01 00:00
Processing 6 candles for BTCUSDT at M interval
   Middlewares: ['levels_middleware']



📊 Visualizing Segment 5/10: BTCUSDT - M
   Period: 2022-03-01 00:00 to 2022-08-01 00:00
Processing 6 candles for BTCUSDT at M interval
   Middlewares: ['levels_middleware']


...


📊 Visualizing Segment 6/10: BTCUSDT - D
   Period: 2022-12-30 00:00 to 2023-07-19 00:00
Processing 202 candles for BTCUSDT at D interval
   Middlewares: ['levels_middleware']



📊 Visualizing Segment 7/10: BTCUSDT - D
   Period: 2023-07-20 00:00 to 2024-02-06 00:00
Processing 202 candles for BTCUSDT at D interval
   Middlewares: ['levels_middleware']



📊 Visualizing Segment 8/10: BTCUSDT - D
   Period: 2024-02-07 00:00 to 2024-08-26 00:00
Processing 202 candles for BTCUSDT at D interval
   Middlewares: ['levels_middleware']



📊 Visualizing Segment 9/10: BTCUSDT - D
   Period: 2024-08-27 00:00 to 2025-03-16 00:00
Processing 202 candles for BTCUSDT at D interval
   Middlewares: ['levels_middleware']



📊 Visualizing Segment 10/10: BTCUSDT - D
   Period: 2025-03-17 00:00 to 2025-10-05 00:00
Processing 203 candles for BTCUSDT at D interval
   Middlewares: ['levels_middleware']
